In [ ]:
from google.colab import files
files.upload()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


!kaggle datasets download -d grassknoted/asl-alphabet

# Extract
import zipfile
import os

datasets = [ 'asl-alphabet.zip']
for dataset in datasets:
    with zipfile.ZipFile(dataset, 'r') as zip_ref:
        zip_ref.extractall('data')
asl_path = "data/asl_alphabet_train/asl_alphabet_train"
selected_classes = ['F', 'G', 'H', 'I', 'R', 'O', 'Q', 'T', 'U', 'V']


In [ ]:
import os
import shutil


selected_classes = ['F', 'G', 'H', 'I', 'R', 'O', 'Q', 'T', 'U', 'V']

# Path to the dataset
asl_train_path = "data/asl_alphabet_train/asl_alphabet_train"

# Create a new folder to store the filtered data
filtered_train_path = 'data/filtered_train'
os.makedirs(filtered_train_path, exist_ok=True)

for class_name in selected_classes:
    class_folder = os.path.join(asl_train_path, class_name)
    if os.path.exists(class_folder):
        target_folder = os.path.join(filtered_train_path, class_name)
        os.makedirs(target_folder, exist_ok=True)

        for image_name in os.listdir(class_folder):
            image_path = os.path.join(class_folder, image_name)
            shutil.copy(image_path, target_folder)


print(f"Filtered dataset stored at: {filtered_train_path}")


In [ ]:
!pip install opencv-python-headless


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import os


transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
full_dataset = datasets.ImageFolder(root='data/filtered_train', transform=transform)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on: {device}")

# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)


num_classes = len(selected_classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Training loop
num_epochs = 10  # You can adjust this based on your requirements

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for inputs, labels in train_loader:
        # Move the data to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track the loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    # Calculate accuracy for the epoch
    epoch_accuracy = 100 * correct_predictions / total_predictions
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Optionally: You can validate the model after each epoch
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # No gradients needed during validation
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

        val_accuracy = 100 * val_correct / val_total
        print(f"Validation Loss: {val_loss / len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%")


In [ ]:
model_path = '/content/drive/My Drive/resnet18_asl_model.pth'

# Save the trained model
torch.save(model.state_dict(), model_path)


In [ ]:
from torchvision import transforms
from PIL import Image
import torch



# تحميل صورة جديدة
image_path = '/content/data/asl_alphabet_train/asl_alphabet_train/F/F1.jpg'  # ضع هنا مسار الصورة
image = Image.open(image_path)

# تطبيق التحويلات على الصورة
image = transform(image).unsqueeze(0)  # إضافة بعد إضافي ليكون الشكل (1, 3, 224, 224)

# التأكد من أن الصورة على نفس الجهاز مثل النموذج (GPU أو CPU)
image = image.to(device)

# ضع النموذج في وضع التقييم
model.eval()

# التنبؤ
with torch.no_grad():  # عدم حساب التدرجات
    outputs = model(image)  # تمرير الصورة عبر النموذج
    _, predicted = torch.max(outputs, 1)  # الحصول على الفئة المتنبأ بها
# معجم من الأرقام إلى الحروف (قم بتعديل هذا حسب ترتيب الفئات في مشروعك)
class_to_label = {
    0: 'F',
    1: 'G',
    2: 'H',
    3: 'I',
    4: 'R',
    5: 'O',
    6: 'Q',
    7: 'T',
    8: 'U',
    9: 'V',
}

# التنبؤ
with torch.no_grad():  # عدم حساب التدرجات
    outputs = model(image)  # تمرير الصورة عبر النموذج
    _, predicted = torch.max(outputs, 1)  # الحصول على الفئة المتنبأ بها

# الحصول على الحرف المتنبأ به
predicted_class = class_to_label[predicted.item()]

# عرض الفئة المتنبأ بها
print(f"Predicted Class: {predicted_class}")


In [ ]:
test_path="data/asl_alphabet_test"
test_dataset=datasets.ImageFolder(root=test_path, transform=transform)
test_loader=DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from torchvision import transforms
from PIL import Image
import torch

# تحميل البيانات المفلترة لمجموعة الاختبار
test_dataset = datasets.ImageFolder(root=test_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# الآن يمكنك اختبار النموذج على هذه البيانات المفلترة كما كنت تفعل سابقًا
model.eval()  # ضع النموذج في وضع التقييم
test_loss = 0.0
test_correct = 0
test_total = 0

# تخزين الصور والعلامات للتنبؤ
images_for_display = []
labels_for_display = []
predictions_for_display = []

with torch.no_grad():  # لا نحتاج لحساب التدرجات أثناء الاختبار
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # التنبؤات
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # التنبؤات الصحيحة
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

        # حفظ بعض الصور لعرضها مع التنبؤات
        images_for_display.extend(inputs.cpu().numpy())
        labels_for_display.extend(labels.cpu().numpy())
        predictions_for_display.extend(predicted.cpu().numpy())

# حساب الدقة على بيانات الاختبار
test_accuracy = 100 * test_correct / test_total
print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")

# عرض بعض الأمثلة مع التنبؤات
def imshow(img):
    img = img / 2 + 0.5  # إلغاء التحجيم
    np_img = img.numpy()
    plt.imshow(np.transpose(np_img, (1, 2, 0)))
    plt.show()

# طباعة بعض الأمثلة
for i in range(5):  # عرض 5 أمثلة
    plt.figure(figsize=(6, 6))
    imshow(torchvision.utils.make_grid(torch.tensor(images_for_display[i])))

    print(f"Actual Label: {test_dataset.classes[labels_for_display[i]]}")
    print(f"Predicted Label: {test_dataset.classes[predictions_for_display[i]]}")
    plt.show()


# Real time _Resnet18

In [ ]:
import cv2
import mediapipe as mp
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import torchvision.models as models


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = torchvision.models.resnet18(pretrained=False)  
model.fc = torch.nn.Linear(in_features=512, out_features=10)

state_dict = torch.load("models/resnet18_asl_model.pth",map_location=torch.device('cpu'))


model.load_state_dict(state_dict, strict=False)

model.eval()

model.to('cpu')

classes = ['F', 'G', 'H', 'I', 'R', 'O', 'Q', 'T', 'U', 'V']

transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            x_coords = [landmark.x * w for landmark in hand_landmarks.landmark]
            y_coords = [landmark.y * h for landmark in hand_landmarks.landmark]
            x_min, x_max = int(min(x_coords)), int(max(x_coords))
            y_min, y_max = int(min(y_coords)), int(max(y_coords))

            margin = 20
            x_min = max(x_min - margin, 0)
            y_min = max(y_min - margin, 0)
            x_max = min(x_max + margin, w)
            y_max = min(y_max + margin, h)

            hand_img = frame[y_min:y_max, x_min:x_max]

            if hand_img.size != 0:
                pil_img = Image.fromarray(cv2.cvtColor(hand_img, cv2.COLOR_BGR2RGB))
                input_tensor = transform(pil_img).unsqueeze(0).to(device)

                with torch.no_grad():
                    output = model(input_tensor)
                    pred = output.argmax(dim=1).item()
                    label = classes[pred]

                cv2.putText(frame, f"Pred: {label}", (x_min, y_min - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("ASL Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
